## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import xml.etree.ElementTree as Xet
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

import seaborn as sns

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:

cols = ["date", "is_holiday"]
rows = []
  
# Parsing the XML file
xmlparse = Xet.parse('/kaggle/input/car-crashes-severity-prediction/holidays.xml')
root = xmlparse.getroot()
for i in root:
    date = i.find("date").text
    is_holiday = i.find("description").text
  
    rows.append({"date": date,
                 "is_holiday": 1,
                 })
  
df1 = pd.DataFrame(rows, columns=cols)
df1.head()

,date,is_holiday
0,2012-01-02,1
1,2012-01-16,1
2,2012-02-20,1
3,2012-05-28,1
4,2012-07-04,1


In [3]:
df1['date'] = pd.to_datetime(df1.date)

In [4]:
(df1['date'].unique()).shape

(90,)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        90 non-null     datetime64[ns]
 1   is_holiday  90 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.5 KB


In [6]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df2 = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df2.shape))

df2.head()

The shape of the dataset is (6901, 12).




,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No


In [7]:
df2['timestamp'] = pd.to_datetime(df2[['Year', 'Month', 'Day', 'Hour']])

In [8]:
df2 = df2.drop(['Year', 'Month', 'Day', 'Hour'], axis=1)

In [9]:

df2

,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,timestamp
0,Fair,64.0,0.00,64.0,70.0,20.0,10.0,No,2020-07-27 18:00:00
1,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-09-30 17:00:00
2,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-06-27 05:00:00
3,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-09-07 09:00:00
4,Fair,52.0,0.00,52.0,89.0,0.0,9.0,No,2019-10-19 02:00:00
...,...,...,...,...,...,...,...,...,...
6896,Clear,NaN,NaN,51.1,80.0,3.5,10.0,No,2018-01-23 21:00:00
6897,Cloudy,56.0,0.00,56.0,80.0,9.0,9.0,No,2019-06-16 07:00:00
6898,Rain,NaN,0.07,61.0,90.0,32.2,7.0,No,2017-02-07 04:00:00
6899,Mostly Cloudy,NaN,NaN,61.0,67.0,21.9,10.0,No,2016-04-22 16:00:00


In [10]:
df2 = df2.drop_duplicates('timestamp')

In [11]:
df2.shape

(5972, 9)

In [12]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df3 = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df3.shape))

df3.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


In [13]:

df3['timestamp'] = pd.to_datetime(df3.timestamp)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            6407 non-null   int64         
 1   Lat           6407 non-null   float64       
 2   Lng           6407 non-null   float64       
 3   Bump          6407 non-null   bool          
 4   Distance(mi)  6407 non-null   float64       
 5   Crossing      6407 non-null   bool          
 6   Give_Way      6407 non-null   bool          
 7   Junction      6407 non-null   bool          
 8   No_Exit       6407 non-null   bool          
 9   Railway       6407 non-null   bool          
 10  Roundabout    6407 non-null   bool          
 11  Stop          6407 non-null   bool          
 12  Amenity       6407 non-null   bool          
 13  Side          6407 non-null   object        
 14  Severity      6407 non-null   int64         
 15  timestamp     6407 non-null   datetime

In [14]:
df3 = df3.assign(Date=df3.timestamp.dt.round('H'))


In [15]:

df3

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp,Date
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02,2016-03-25 15:00:00
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00,2020-05-05 19:00:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16,2016-09-16 20:00:00
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43,2020-03-29 20:00:00
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00,2019-10-09 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,False,0.368,False,False,False,False,False,False,False,False,R,3,2017-10-01 18:36:13,2017-10-01 19:00:00
6403,6403,37.752755,-122.402790,False,0.639,False,False,True,False,False,False,False,False,R,2,2018-10-23 07:40:27,2018-10-23 08:00:00
6404,6404,37.726304,-122.446015,False,0.000,False,False,True,False,False,False,False,False,R,2,2019-10-28 15:45:00,2019-10-28 16:00:00
6405,6405,37.808090,-122.367211,False,0.000,False,False,True,False,False,False,False,False,R,3,2019-05-04 13:45:31,2019-05-04 14:00:00


In [16]:
df_df3anddf2 = pd.merge(df3, df2, left_on='Date', right_on='timestamp', how='left')
df_df3anddf2.shape

(6407, 26)

In [17]:

for col in df_df3anddf2.columns:
    print(col)
    print("------------------------")
    print(df_df3anddf2[col].unique())
    print("------------------------")

ID
------------------------
[   0    1    2 ... 6404 6405 6406]
------------------------
Lat
------------------------
[37.76215  37.719157 37.808498 ... 37.756012 37.788067 37.80809 ]
------------------------
Lng
------------------------
[-122.40566  -122.448254 -122.366852 ... -122.403885 -122.440445
 -122.367211]
------------------------
Bump
------------------------
[False]
------------------------
Distance(mi)
------------------------
[4.40000000e-02 0.00000000e+00 9.00000000e-03 1.00000000e-02
 5.79999983e-01 1.98000000e-01 8.57000000e-01 5.40000021e-01
 4.42000000e-01 3.00000012e-01 1.20000000e-02 5.39000000e-01
 1.36000000e-01 2.49000000e-01 2.75000000e-01 8.00000000e-03
 3.74000000e-01 1.06000000e-01 6.70000000e-01 4.22000000e-01
 4.06000000e-01 6.11000000e-01 2.70000011e-01 1.00000000e-03
 2.14000000e-01 5.17000000e-01 6.90000000e-02 5.70000000e-02
 3.70000000e-01 7.44000000e-01 1.12000000e-01 7.80000000e-02
 1.31000000e-01 4.72000000e-01 6.74000000e-01 3.50000000e-02
 3.70000

In [18]:
df_df3anddf2 = df_df3anddf2.drop(columns=['Bump', 'Roundabout', 'Selected', 'No_Exit'])

In [19]:
meanWind_Chill = df_df3anddf2['Wind_Chill(F)'].median()

df_df3anddf2['Wind_Chill(F)'] = df_df3anddf2['Wind_Chill(F)'].fillna(meanWind_Chill)

In [20]:
meanPrecipitation = df_df3anddf2['Precipitation(in)'].median()

df_df3anddf2['Precipitation(in)'] = df_df3anddf2['Precipitation(in)'].fillna(meanPrecipitation)

In [21]:


meanWind_Speed = df_df3anddf2['Wind_Speed(mph)'].median()

df_df3anddf2['Wind_Speed(mph)'] = df_df3anddf2['Wind_Speed(mph)'].fillna(meanWind_Speed)

In [22]:


df_df3anddf2.isnull().sum()

ID                      0
Lat                     0
Lng                     0
Distance(mi)            0
Crossing                0
Give_Way                0
Junction                0
Railway                 0
Stop                    0
Amenity                 0
Side                    0
Severity                0
timestamp_x             0
Date                    0
Weather_Condition    2247
Wind_Chill(F)           0
Precipitation(in)       0
Temperature(F)       2247
Humidity(%)          2247
Wind_Speed(mph)         0
Visibility(mi)       2247
timestamp_y          2246
dtype: int64

-----------------------

In [23]:

meanTemperature = df_df3anddf2['Temperature(F)'].median()

df_df3anddf2['Temperature(F)'] = df_df3anddf2['Temperature(F)'].fillna(meanTemperature)

In [24]:
meanVisibility  = df_df3anddf2['Visibility(mi)'].median()

df_df3anddf2['Visibility(mi)'] = df_df3anddf2['Visibility(mi)'].fillna(meanVisibility)

In [25]:

meanHumidity = df_df3anddf2['Humidity(%)'].median()

df_df3anddf2['Humidity(%)'] = df_df3anddf2['Humidity(%)'].fillna(meanHumidity)

----------------

In [26]:

df_df3anddf2["Crossing"] = df_df3anddf2["Crossing"].astype(int)
df_df3anddf2["Give_Way"] = df_df3anddf2["Give_Way"].astype(int)
df_df3anddf2["Junction"] = df_df3anddf2["Junction"].astype(int)
df_df3anddf2["Railway"] = df_df3anddf2["Railway"].astype(int)
df_df3anddf2["Stop"] = df_df3anddf2["Stop"].astype(int)
df_df3anddf2["Amenity"] = df_df3anddf2["Amenity"].astype(int)



In [27]:
cols_categ_encoding = ['Side', 'Weather_Condition']

In [28]:

df_df3anddf2.Weather_Condition = df_df3anddf2.Weather_Condition.astype('str')

In [29]:
df_df3anddf2_new = df_df3anddf2.copy()
my_encoder = LabelEncoder()

for col in cols_categ_encoding:
    df_df3anddf2_new[col] = my_encoder.fit_transform(df_df3anddf2_new[col])
    

In [30]:


df_df3anddf2_new = df_df3anddf2_new.fillna(df_df3anddf2_new['Weather_Condition'].value_counts().index[0])

In [31]:
df_df3anddf2_new.isnull().sum()


ID                   0
Lat                  0
Lng                  0
Distance(mi)         0
Crossing             0
Give_Way             0
Junction             0
Railway              0
Stop                 0
Amenity              0
Side                 0
Severity             0
timestamp_x          0
Date                 0
Weather_Condition    0
Wind_Chill(F)        0
Precipitation(in)    0
Temperature(F)       0
Humidity(%)          0
Wind_Speed(mph)      0
Visibility(mi)       0
timestamp_y          0
dtype: int64



We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [32]:
df_df3anddf2_new

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,...,timestamp_x,Date,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),timestamp_y
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,1,...,2016-03-25 15:13:02,2016-03-25 15:00:00,19,59.0,0.0,64.0,58.0,23.0,10.0,2016-03-25 15:00:00
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,2020-05-05 19:23:00,2020-05-05 19:00:00,13,57.0,0.0,57.0,83.0,22.0,10.0,2020-05-05 19:00:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,1,0,...,2016-09-16 19:57:16,2016-09-16 20:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,...,2020-03-29 19:48:43,2020-03-29 20:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,...,2019-10-09 08:47:00,2019-10-09 09:00:00,3,59.0,0.0,59.0,75.0,0.0,10.0,2019-10-09 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,...,2017-10-01 18:36:13,2017-10-01 19:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
6403,6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,...,2018-10-23 07:40:27,2018-10-23 08:00:00,12,59.0,0.0,55.0,77.0,4.6,10.0,2018-10-23 08:00:00
6404,6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,...,2019-10-28 15:45:00,2019-10-28 16:00:00,3,67.0,0.0,67.0,16.0,15.0,10.0,2019-10-28 16:00:00
6405,6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,...,2019-05-04 13:45:31,2019-05-04 14:00:00,16,63.0,0.0,63.0,54.0,22.0,10.0,2019-05-04 14:00:00


In [33]:
df_df3anddf2_new.corr()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,Side,Severity,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
ID,1.000000,0.012043,0.009135,0.024395,0.010125,-0.021899,0.007417,0.004945,0.003892,0.003176,0.007809,0.020703,0.010911,0.000169,0.004173,-0.014060,0.009494,0.003568,-0.022873
Lat,0.012043,1.000000,0.388177,0.039760,0.040971,0.011296,0.012027,-0.002388,0.390530,0.088219,0.029349,0.099581,-0.013941,0.021056,-0.038343,0.033140,0.004264,-0.033337,-0.004673
Lng,0.009135,0.388177,1.000000,0.074003,-0.123170,-0.032626,0.191309,-0.030877,0.385575,-0.102261,0.082084,0.145313,-0.015601,0.002832,-0.013593,0.007191,0.011337,-0.047278,-0.014737
Distance(mi),0.024395,0.039760,0.074003,1.000000,-0.020309,-0.007164,-0.028275,-0.033987,-0.054533,-0.033825,0.002264,-0.013141,-0.002289,-0.009847,-0.016794,-0.012588,0.002541,-0.024809,0.011448
Crossing,0.010125,0.040971,-0.123170,-0.020309,1.000000,0.072222,-0.160848,0.430823,-0.037446,0.319284,-0.256629,-0.090314,0.015864,-0.027293,-0.021616,-0.032018,0.007458,-0.022656,0.014669
Give_Way,-0.021899,0.011296,-0.032626,-0.007164,0.072222,1.000000,-0.012378,0.041317,0.041475,-0.004251,0.005003,-0.012186,-0.004663,0.004963,-0.003219,0.003135,-0.015112,-0.004070,0.005868
Junction,0.007417,0.012027,0.191309,-0.028275,-0.160848,-0.012378,1.000000,-0.094416,0.075290,-0.089347,0.123946,-0.068328,-0.026036,0.024169,0.022663,0.018271,-0.028549,0.003395,-0.015813
Railway,0.004945,-0.002388,-0.030877,-0.033987,0.430823,0.041317,-0.094416,1.000000,-0.034703,0.126759,-0.116734,-0.033322,-0.012418,-0.002881,-0.008958,-0.000192,-0.023146,-0.001020,0.001051
Stop,0.003892,0.390530,0.385575,-0.054533,-0.037446,0.041475,0.075290,-0.034703,1.000000,-0.039619,0.040135,0.229269,-0.007570,0.006552,-0.015799,0.004251,0.016905,-0.031742,-0.022261
Amenity,0.003176,0.088219,-0.102261,-0.033825,0.319284,-0.004251,-0.089347,0.126759,-0.039619,1.000000,-0.247947,-0.078915,0.008156,-0.003218,-0.016942,0.002857,-0.003807,0.007495,0.012890


In [34]:
df_df3anddf2_new.describe()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,Side,Severity,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6407.000000,6407.000000,6407.000000,6407.000000,6407.00000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000
mean,3203.000000,37.765653,-122.405990,0.135189,0.08241,0.000468,0.246449,0.026533,0.097706,0.037147,0.949274,2.293429,13.963165,59.425628,0.002500,59.736866,68.222413,10.637802,9.617492
std,1849.685919,0.032555,0.028275,0.396360,0.27501,0.021635,0.430977,0.160728,0.296939,0.189136,0.219454,0.521225,7.565876,5.412166,0.016812,6.420685,13.209780,5.165131,1.411027
min,0.000000,37.609619,-122.510440,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,34.000000,0.000000,37.900000,10.000000,0.000000,0.120000
25%,1601.500000,37.737096,-122.412210,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,9.000000,59.000000,0.000000,57.000000,65.000000,8.100000,10.000000
50%,3203.000000,37.768238,-122.404835,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,15.000000,59.000000,0.000000,59.000000,69.000000,10.000000,10.000000
75%,4804.500000,37.787813,-122.392477,0.041000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,22.000000,59.000000,0.000000,62.000000,75.000000,12.700000,10.000000
max,6406.000000,37.825626,-122.349734,6.820000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,22.000000,98.000000,0.490000,98.000000,100.000000,37.000000,10.000000


In [35]:

df_df3anddf2_new['timestamp_x'] = df_df3anddf2_new['timestamp_x'].dt.date
df_df3anddf2_new['timestamp_x'] = pd.to_datetime(df_df3anddf2_new.timestamp_x)

In [36]:
df_df3anddf2_new

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,...,timestamp_x,Date,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),timestamp_y
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,1,...,2016-03-25,2016-03-25 15:00:00,19,59.0,0.0,64.0,58.0,23.0,10.0,2016-03-25 15:00:00
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,2020-05-05,2020-05-05 19:00:00,13,57.0,0.0,57.0,83.0,22.0,10.0,2020-05-05 19:00:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,1,0,...,2016-09-16,2016-09-16 20:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,...,2020-03-29,2020-03-29 20:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,...,2019-10-09,2019-10-09 09:00:00,3,59.0,0.0,59.0,75.0,0.0,10.0,2019-10-09 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,...,2017-10-01,2017-10-01 19:00:00,22,59.0,0.0,59.0,69.0,10.0,10.0,22
6403,6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,...,2018-10-23,2018-10-23 08:00:00,12,59.0,0.0,55.0,77.0,4.6,10.0,2018-10-23 08:00:00
6404,6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,...,2019-10-28,2019-10-28 16:00:00,3,67.0,0.0,67.0,16.0,15.0,10.0,2019-10-28 16:00:00
6405,6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,...,2019-05-04,2019-05-04 14:00:00,16,63.0,0.0,63.0,54.0,22.0,10.0,2019-05-04 14:00:00


In [37]:
df_df1andmerge = pd.merge(df_df3anddf2_new, df1, left_on='timestamp_x', right_on='date', how='left')

In [38]:
df_df1andmerge['is_holiday'] = df_df1andmerge['is_holiday'].fillna(0)

In [39]:
df_df1andmerge['is_holiday'].unique()

array([0., 1.])

In [40]:
df_df1andmerge

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,...,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),timestamp_y,date,is_holiday
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,1,...,19,59.0,0.0,64.0,58.0,23.0,10.0,2016-03-25 15:00:00,NaT,0.0
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,13,57.0,0.0,57.0,83.0,22.0,10.0,2020-05-05 19:00:00,NaT,0.0
2,2,37.808498,-122.366852,0.000,0,0,0,0,1,0,...,22,59.0,0.0,59.0,69.0,10.0,10.0,22,NaT,0.0
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,...,22,59.0,0.0,59.0,69.0,10.0,10.0,22,NaT,0.0
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,...,3,59.0,0.0,59.0,75.0,0.0,10.0,2019-10-09 09:00:00,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,...,22,59.0,0.0,59.0,69.0,10.0,10.0,22,NaT,0.0
6403,6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,...,12,59.0,0.0,55.0,77.0,4.6,10.0,2018-10-23 08:00:00,NaT,0.0
6404,6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,...,3,67.0,0.0,67.0,16.0,15.0,10.0,2019-10-28 16:00:00,NaT,0.0
6405,6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,...,16,63.0,0.0,63.0,54.0,22.0,10.0,2019-05-04 14:00:00,NaT,0.0


In [41]:
df_df1andmerge.corr()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,Side,Severity,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),is_holiday
ID,1.000000,0.012043,0.009135,0.024395,0.010125,-0.021899,0.007417,0.004945,0.003892,0.003176,0.007809,0.020703,0.010911,0.000169,0.004173,-0.014060,0.009494,0.003568,-0.022873,0.006486
Lat,0.012043,1.000000,0.388177,0.039760,0.040971,0.011296,0.012027,-0.002388,0.390530,0.088219,0.029349,0.099581,-0.013941,0.021056,-0.038343,0.033140,0.004264,-0.033337,-0.004673,-0.009443
Lng,0.009135,0.388177,1.000000,0.074003,-0.123170,-0.032626,0.191309,-0.030877,0.385575,-0.102261,0.082084,0.145313,-0.015601,0.002832,-0.013593,0.007191,0.011337,-0.047278,-0.014737,-0.028472
Distance(mi),0.024395,0.039760,0.074003,1.000000,-0.020309,-0.007164,-0.028275,-0.033987,-0.054533,-0.033825,0.002264,-0.013141,-0.002289,-0.009847,-0.016794,-0.012588,0.002541,-0.024809,0.011448,0.009794
Crossing,0.010125,0.040971,-0.123170,-0.020309,1.000000,0.072222,-0.160848,0.430823,-0.037446,0.319284,-0.256629,-0.090314,0.015864,-0.027293,-0.021616,-0.032018,0.007458,-0.022656,0.014669,0.018149
Give_Way,-0.021899,0.011296,-0.032626,-0.007164,0.072222,1.000000,-0.012378,0.041317,0.041475,-0.004251,0.005003,-0.012186,-0.004663,0.004963,-0.003219,0.003135,-0.015112,-0.004070,0.005868,-0.003328
Junction,0.007417,0.012027,0.191309,-0.028275,-0.160848,-0.012378,1.000000,-0.094416,0.075290,-0.089347,0.123946,-0.068328,-0.026036,0.024169,0.022663,0.018271,-0.028549,0.003395,-0.015813,0.001267
Railway,0.004945,-0.002388,-0.030877,-0.033987,0.430823,0.041317,-0.094416,1.000000,-0.034703,0.126759,-0.116734,-0.033322,-0.012418,-0.002881,-0.008958,-0.000192,-0.023146,-0.001020,0.001051,0.006937
Stop,0.003892,0.390530,0.385575,-0.054533,-0.037446,0.041475,0.075290,-0.034703,1.000000,-0.039619,0.040135,0.229269,-0.007570,0.006552,-0.015799,0.004251,0.016905,-0.031742,-0.022261,0.005387
Amenity,0.003176,0.088219,-0.102261,-0.033825,0.319284,-0.004251,-0.089347,0.126759,-0.039619,1.000000,-0.247947,-0.078915,0.008156,-0.003218,-0.016942,0.002857,-0.003807,0.007495,0.012890,0.019241


In [42]:
df_df1andmerge['Severity'].value_counts()

2    4346
3    1855
1     129
4      77
Name: Severity, dtype: int64

In [43]:
i=0
for ind,row in df_df1andmerge.iterrows():
    if row['Severity']==2 and i<3000:
        df_df1andmerge=df_df1andmerge.drop(ind)
        i+=1

In [44]:
i=0
for ind,row in df_df1andmerge.iterrows():
    if row['Severity']==3 and i<1000:
        df_df1andmerge=df_df1andmerge.drop(ind)
        i+=1

In [45]:
df_df1andmerge['Severity'].value_counts()

2    1346
3     855
1     129
4      77
Name: Severity, dtype: int64

In [46]:
df_df1andmerge.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,Railway,Stop,Amenity,...,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),timestamp_y,date,is_holiday
3,3,37.78593,-122.39108,0.009,0,0,1,0,0,0,...,22,59.0,0.00,59.0,69.0,10.0,10.0,22,NaT,0.0
12,12,37.81053,-122.36464,0.000,0,0,1,0,0,0,...,22,59.0,0.00,59.0,69.0,10.0,10.0,22,NaT,0.0
14,14,37.73007,-122.43918,0.857,0,0,0,0,0,0,...,9,53.0,0.11,53.0,83.0,10.0,4.0,2020-04-05 11:00:00,NaT,0.0
52,52,37.74496,-122.40522,0.000,0,0,1,0,0,0,...,3,52.0,0.00,52.0,83.0,14.0,10.0,2020-06-07 05:00:00,NaT,0.0
60,60,37.74497,-122.40521,0.000,0,0,1,0,0,0,...,13,53.0,0.00,53.0,80.0,23.0,10.0,2020-04-30 23:00:00,NaT,0.0


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [47]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_df1andmerge, test_size=0.2, random_state=42) 
# Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [48]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train.drop(columns=['Distance(mi)', 'Wind_Chill(F)', 'date', 'timestamp_y', 'timestamp_x', 'Date'])
X_val = X_val.drop(columns=['Distance(mi)', 'Wind_Chill(F)', 'date', 'timestamp_y', 'timestamp_x', 'Date'])

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [49]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [50]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))


The accuracy of the classifier on the validation set is  0.6680497925311203


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [51]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


In [52]:
test_df.shape

(1601, 15)

In [53]:
test_df['timestamp'] = pd.to_datetime(test_df.timestamp)
test_df = test_df.assign(Date=test_df.timestamp.dt.round('H'))
test_df

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp,Date
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31,2016-04-04 19:00:00
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00,2020-10-28 12:00:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45,2019-09-09 08:00:00
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25,2019-08-06 16:00:00
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58,2018-10-17 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,False,4.460,False,False,False,False,False,False,False,False,R,2020-06-26 22:32:22,2020-06-26 23:00:00
1597,8004,37.761818,-122.405861,False,0.010,False,False,True,False,False,False,False,False,R,2016-12-03 07:16:30,2016-12-03 07:00:00
1598,8005,37.732260,-122.431970,False,0.431,False,False,True,False,False,False,False,False,R,2017-02-20 06:32:44,2017-02-20 07:00:00
1599,8006,37.786782,-122.390126,False,0.000,True,False,False,False,False,False,False,False,R,2019-10-31 20:35:00,2019-10-31 21:00:00


In [54]:
df_test_dfanddf2 = pd.merge(test_df, df2, left_on='Date', right_on='timestamp', how='left')

In [55]:
df_test_dfanddf2.shape

(1601, 25)

In [56]:
df_test_dfanddf2 = df_test_dfanddf2.drop(columns=['Bump', 'Roundabout', 'Selected', 'No_Exit'])



meanWind_Chill = df_test_dfanddf2['Wind_Chill(F)'].median()

df_test_dfanddf2['Wind_Chill(F)'] = df_test_dfanddf2['Wind_Chill(F)'].fillna(meanWind_Chill)

meanPrecipitation = df_test_dfanddf2['Precipitation(in)'].median()

df_test_dfanddf2['Precipitation(in)'] = df_test_dfanddf2['Precipitation(in)'].fillna(meanPrecipitation)

meanWind_Speed = df_test_dfanddf2['Wind_Speed(mph)'].median()

df_test_dfanddf2['Wind_Speed(mph)'] = df_test_dfanddf2['Wind_Speed(mph)'].fillna(meanWind_Speed)


meanTemperature = df_test_dfanddf2['Temperature(F)'].median()

df_test_dfanddf2['Temperature(F)'] = df_test_dfanddf2['Temperature(F)'].fillna(meanTemperature)

meanVisibility  = df_test_dfanddf2['Visibility(mi)'].median()

df_test_dfanddf2['Visibility(mi)'] = df_test_dfanddf2['Visibility(mi)'].fillna(meanVisibility)

meanHumidity = df_test_dfanddf2['Humidity(%)'].median()

df_test_dfanddf2['Humidity(%)'] = df_test_dfanddf2['Humidity(%)'].fillna(meanHumidity)


df_test_dfanddf2["Crossing"] = df_test_dfanddf2["Crossing"].astype(int)
df_test_dfanddf2["Give_Way"] = df_test_dfanddf2["Give_Way"].astype(int)
df_test_dfanddf2["Junction"] = df_test_dfanddf2["Junction"].astype(int)
df_test_dfanddf2["Railway"] = df_test_dfanddf2["Railway"].astype(int)
df_test_dfanddf2["Stop"] = df_test_dfanddf2["Stop"].astype(int)
df_test_dfanddf2["Amenity"] = df_test_dfanddf2["Amenity"].astype(int)

cols_categ_encoding = ['Side', 'Weather_Condition']

df_test_dfanddf2.Weather_Condition = df_test_dfanddf2.Weather_Condition.astype('str')


df_test_dfanddf2_new = df_test_dfanddf2.copy()
my_encoder = LabelEncoder()

for col in cols_categ_encoding:
    df_test_dfanddf2_new[col] = my_encoder.fit_transform(df_test_dfanddf2_new[col])
    

df_df3anddf2_new = df_df3anddf2_new.fillna(df_df3anddf2_new['Weather_Condition'].value_counts().index[0])


df_df3anddf2_new['timestamp_x'] = df_df3anddf2_new['timestamp_x'].dt.date
df_df3anddf2_new['timestamp_x'] = pd.to_datetime(df_df3anddf2_new.timestamp_x)

df_df1andmerge = pd.merge(df_df3anddf2_new, df1, left_on='timestamp_x', right_on='date', how='left')


df_df1andmerge['is_holiday'] = df_df1andmerge['is_holiday'].fillna(0)




In [57]:
df_test_dfanddf2_new.shape

(1601, 21)

In [58]:
X_test = df_test_dfanddf2_new

# You should update/remove the next line once you change the features used for training
X_test = X_test.drop(columns=['Distance(mi)', 'Wind_Chill(F)', 'timestamp_y', 'timestamp_x', 'Date'])
X_test

,ID,Lat,Lng,Crossing,Give_Way,Junction,Railway,Stop,Amenity,Side,Weather_Condition,Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,6407,37.786060,-122.390900,0,0,1,0,0,0,1,14,0.0,63.0,60.0,10.4,10.0
1,6408,37.769609,-122.415057,0,0,0,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0
2,6409,37.807495,-122.476021,0,0,0,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0
3,6410,37.761818,-122.405869,0,0,1,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0
4,6411,37.732350,-122.414100,0,0,0,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,0,0,0,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0
1597,8004,37.761818,-122.405861,0,0,1,0,0,0,1,14,0.0,46.0,66.0,10.0,10.0
1598,8005,37.732260,-122.431970,0,0,1,0,0,0,1,13,0.0,57.9,84.0,9.2,10.0
1599,8006,37.786782,-122.390126,1,0,0,0,0,0,1,21,0.0,60.0,70.0,10.0,10.0


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [59]:
X_test = df_test_dfanddf2_new

# You should update/remove the next line once you change the features used for training
X_test = X_test.drop(columns=['Distance(mi)', 'Wind_Chill(F)', 'timestamp_y', 'timestamp_x', 'Date'])

y_test_predicted = classifier.predict(X_test)

df_test_dfanddf2_new['Severity'] = y_test_predicted

df_test_dfanddf2_new["ID"] = df_test_dfanddf2_new["ID"].astype(int)
df_test_dfanddf2_new["Severity"] = df_test_dfanddf2_new["Severity"].astype(int)

df_test_dfanddf2_new['Severity'].unique()


array([2, 3])

In [60]:
df_test_dfanddf2_new.shape

(1601, 22)

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [61]:
df_test_dfanddf2_new[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.